# Curricular Analytics - What if? Simulator

Hi! This notebook is here to walk you through simulating changes to a curriculum. You give us the file and the desired change and we'll tell you how that affects complexity and centrality, and why. There are animated examples for each section. If there's an error you don't understand, or you want to suggest improvements, please contact [a1amaya@ucsd.edu](mailto:a1amaya@ucsd.edu). But first, let us do some basic-set up. Run the cell below to do that: (if you've never used a Jupyter Notebook, press ctrl+Enter or cmd+Enter to run a selected cell)

In [1]:
using CurricularAnalytics, CurricularAnalyticsDiff

Ok! We're all set! There are four modifications you can make: you can add a course, remove a course, add a prereq and remove a prereq. After adding the curriculum you wish to change by running the next cell, just scroll on down to the section you want to work with. Currently, we'll be adding the FILENAME file in the targets folder. Feel free to upload your own curriculum file and change the file path.

<img src="./assets/SelectAFile.gif" />

In [ ]:
curr = read_csv("./targets/FILENAME.csv");
# dont' worry about this, this is just to make sure we're working with curricula, not degree plans
if typeof(curr) == DegreePlan
    curr = curr.curriculum
end

# Make modifications

## Add a Course:


Let's add a course. A course is defined by a few things. It has a name, credit_hours, and prerequisites. It also usually has a list of classes that depend on it. i.e. the classes that have it as a prerequisite. Let's define those things right now. 

NOTE:
When editing the prereqs field, make sure the "COURSE NAME #" field is replaced by the name of the existing course in the target curriculum that is to become a prereq of your new course. The same concept applies to the "COURSE NAME #" in the dependencies field. Once you're done adding prerequisites and dependencies, delete the extra fields.
Pre refers to the type of prerequisite. For now, let's leave that alone.

As an example, let's try to add MATH 20C to a curriculum.

new_course_name = "MATH 20C"

new_course_credit_hours = 4.0

prereqs = Dict("MATH 18" => pre,
               "MATH 20B" => pre)
               
dependencies = Dict("MATH 20D" => pre, "MATH 20E" => pre)

Note that in a curriculum where MATH 20B is called "MATH 20B/10B" or something like that you must use the name used in the curriculum file, otherwise this will NOT work. 

<img src="./assets/AddACourse.gif" />

In [ ]:
new_course_name = "NEW COURSE NAME";
new_course_credit_hours = 4.0; # defualt, you can change it to 1.0,2.0,3.0, etc
prereqs = Dict("PREREQ 1" => pre,
               "PREREQ 2" => pre,
               "PREREQ 3" => pre);
dependencies = Dict("DEPENDENT 1" => pre,
                    "DEPENDENT 2" => pre,
                    "DEPENDENT 3" => pre);

Run the above cell once you're done, then run this one:

In [ ]:
new_curric = add_course(curr, new_course_name, new_course_credit_hours, prereqs, dependencies);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

If you're getting an error, please make sure all the course names you're using match the ones in the file exactly.

## Remove a course

Removing a course is pretty easy. Just tell us the name of the course to be removed in the field below. Remember, it must match exactly with the name used in the curriculum file. 

<img src="./assets/RemoveACourse.gif" style="margin: -20px 0px -150px 0px"/>

In [ ]:
course_to_remove = "COURSE NAME";

Then, run this cell:

In [ ]:
new_curric = remove_course(curr, course_to_remove);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

## Add a prerequisite

To add a prerequisite to a course tell us the name of the course, and the name of the prerequisite you wish to add to it. Remember, both names must already exist in the curriculum and must match exactly with your names. For example, if MATH 20B shows up as "MATH 20B/10B" in your curriculum, use that.

<img src="./assets/AddAPrereq.gif" />

In [ ]:
course_name = "COURSE TO GET A PREREQUISITE";
prerequisite = "PREREQ NAME";
req_type = pre;

Once you're ready, run this cell:

In [ ]:
new_curric = add_prereq(curr, course_name, prerequisite,req_type);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

## Remove a prerequisite

Removing a prerequisite needs the name of the course in question, and the name of the course which we want to remove as its prerequisite. Remember the names need to match the ones used in the curriculum file exactly.

<img src="./assets/RemoveAPrereq.gif" />

In [ ]:
course_name = "COURSE TO LOSE A PREREQUISITE";
prerequisite = "PREREQUISITE TO BE LOST";

All good? Run the cell below

In [ ]:
new_curric = remove_prereq(curr,course_name,prerequisite);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

# How did that affect the Curricular Metrics?

This is an easy question to answer. Just run the cell below after having made a new_curric with one of the previous modifications

In [ ]:
results = curricular_diff(curr,new_curric);

Once you're done with that, choose how you want to see your results: executive summary or pretty print

In [ ]:
executive_summary_curriculum(results)

In [ ]:
pretty_print_curriculum_results(results, ALL)

# Wrapping up

You can play around with this file, and you can write your new curriculum to file with the following cell. Unfortunately, we can't visualize the curricula here in this notebook, but you can take them over to the curricular analytics website to view.

<img src="./assets/DownloadAFile.gif" />

In [ ]:
write_csv(new_curric, "./newfile.csv")

The full package is available to you now to use. If you wish to make more complex modifications, be our guest! With these very basic building blocks you can make very complex edits to curricula.

# UCSD Specific:

This is slightly more experimental. Let's see how deleting a prerequisite can affect several majors.

In [3]:
UCSD = read_csv("./targets/condensed.csv");
isvalid_curriculum(UCSD,IOBuffer())

true

In [ ]:
target = "TARGET NAME";
prereq = "PREREQ NAME";
target_course = course_from_name(UCSD,target);
prereq_course = course_from_name(UCSD,prereq);

In [ ]:
delete_requisite!(prereq_course,target_course)
# if it's all good:\n,
target_course_majors = split(target_course.canonical_name,",");
prereq_course_majors = split(prereq_course.canonical_name,",");
prev_major = "PL99";
count = 0;
for major in target_course_majors
    if major in prereq_course_majors
        #println("\n$major")
        if major != "" 
            if major[1:4] != prev_major[1:4]
                prev_major = major
                print("\n$(major[1:4]): $(major[5:end]), ");
                count += 1
            elseif major != prev_major
                prev_major = major
                print("$(major[5:end]), ");
                count += 1
            end
        end
    end
end
println()
print("Affected plans: $count")

Ok, removing a course should be pretty simple

In [ ]:
UCSD = read_csv("./targets/condensed.csv");
course_to_remove_name = "COURSE NAME"
course_to_remove = course_from_name(UCSD,course_to_remove_name);

In [ ]:
affected_majors = split(course_to_remove.canonical_name,",");
prev_major = "PL99";
count = 0;
for major in affected_majors
    if major != "" 
        if major[1:4] != prev_major[1:4]
            prev_major = major
            print("\n$(major[1:4]): $(major[5:end]), ");
            count += 1
        elseif major != prev_major
            prev_major = major
            print("$(major[5:end]), ");
            count += 1
        end
    end
end
println()
print("Affected plans: $count")
# NOTE THIS DOESNT ACTUALLY CHANGE THE CURRICULUM OBJECT OK?

Note that printing the majors that class is associated with should be enough. Checking all the classes that depend on itor it depends on *should* be redundant - if that class is linked to this one explicitly in a major they should both appear in that curriculum. Thus that major should appear in this class's canonical name. TODO proof.

Next, we should talk about adding classes or adding prerequisites. This is slightly more complicated. A new class doesn't have any majors to be associated with inherently (altough that's an option for the future) so we need to hook it up and then check all of its dependents and all of its prereqs. TODO: formal proof.

In [ ]:
UCSD = read_csv("./targets/condensed.csv");
new_course_name = "NEW COURSE NAME";
new_course_credit_hours = 4.0; # defualt, you can change it to 1.0,2.0,3.0, etc
prereqs = Dict("PREREQ 1" => pre,
               "PREREQ 2" => pre,
               "PREREQ 3" => pre);
dependencies = Dict("DEPENDENT 1" => pre,
                    "DEPENDENT 2" => pre,
                    "DEPENDENT 3" => pre);
new_UCSD = add_course(UCSD, new_course_name, new_course_credit_hours, prereqs, dependencies);
errors = IOBuffer();
isvalid_curriculum(new_UCSD,errors)

In [7]:
# get all the paths that depend on me
## first, get me
#UCSD = read_csv("./targets/condensed.csv");
course = course_from_name(new_UCSD,new_course_name);
my_centrality_paths = centrality_investigator(course,new_UCSD);
if length(my_centrality_paths) > 0
    # ok actually do stuff
    # the gist is:
    # look at all the paths that I'm a prereq for and for each path take the intersection of their majors
    ## get all the paths that depend on me:
    prereq_set = Set()
    dep_set = Set()
    for path in my_centrality_paths
        my_index = findall(x->x==course,path)[1]
        # course is path[my_index]
        # TODO: edge cases based on length
        my_prereqs = path[1:my_index-1]
        my_deps = path[my_index+1:end]
        #= HUGE EDIT: only analyze the dependencies
        path_set = Set()
        for prereq in my_prereqs
            if isempty(path_set)
                path_set = Set(split(prereq.canonical_name,","))
            else
                intersect!(path_set,Set(split(prereq.canonical_name,",")))
            end
        end
        union!(prereq_set,path_set)
        =#
        path_set=Set()
        for dep in my_deps
            if isempty(path_set)
                path_set = Set(split(dep.canonical_name,","))
            else
                union!(path_set,Set(split(dep.canonical_name,",")))
            end
        end
        union!(dep_set,path_set)
        
    end
    full_set = union(prereq_set,dep_set)
    full_set = sort(collect(full_set))
    print(full_set)           
    # look at all the paths that depend on me and for each path take the union of their majors
    # then combine the two sets
else
    # ok this seems to not affect any majors because it's not been hooked up to anything
    println("This course hasn't been hooked up to anything, it didn't affect any majors other than the one it is in");
end

Any["", "BE25FI", "BE25MU", "BE25RE", "BE25SI", "BE25SN", "BE25TH", "BE25WA", "BE25curriculum", "BE27FI", "BE27MU", "BE27RE", "BE27SI", "BE27SN", "BE27TH", "BE27WA", "BE27curriculum", "BE28FI", "BE28MU", "BE28RE", "BE28SI", "BE28SN", "BE28TH", "BE28WA", "BE28curriculum", "BE29FI", "BE29MU", "BE29RE", "BE29SI", "BE29SN", "BE29TH", "BE29WA", "BE29curriculum", "BI34FI", "BI34MU", "BI34RE", "BI34SI", "BI34SN", "BI34TH", "BI34WA", "BI34curriculum", "CE25FI", "CE25MU", "CE25RE", "CE25SI", "CE25SN", "CE25TH", "CE25WA", "CE25curriculum", "CH25FI", "CH25MU", "CH25RE", "CH25SI", "CH25SN", "CH25TH", "CH25WA", "CH25curriculum", "CH34FI", "CH34MU", "CH34RE", "CH34SI", "CH34SN", "CH34TH", "CH34WA", "CH34curriculum", "CH35FI", "CH35MU", "CH35RE", "CH35SI", "CH35SN", "CH35TH", "CH35WA", "CH35curriculum", "CH36FI", "CH36MU", "CH36RE", "CH36SI", "CH36SN", "CH36TH", "CH36WA", "CH36curriculum", "CH38FI", "CH38MU", "CH38RE", "CH38SI", "CH38SN", "CH38TH", "CH38WA", "CH38curriculum", "CS25FI", "CS25MU", "CS2

In [13]:
# what is in the 20c canon name but not in the calculated set
sort(collect(setdiff(Set(split(course.canonical_name,",")),full_set)))

16-element Vector{SubString{String}}:
 "CS26FI"
 "CS26MU"
 "CS26RE"
 "CS26SI"
 "CS26SN"
 "CS26TH"
 "CS26WA"
 "CS26curriculum"
 "PB26FI"
 "PB26MU"
 "PB26RE"
 "PB26SI"
 "PB26SN"
 "PB26TH"
 "PB26WA"
 "PB26curriculum"

In [16]:
# what is in the full set but not in the 20c canon name
for val in sort(collect(setdiff(full_set,Set(split(course.canonical_name,",")))))
    println(val)
end

BE27FI
BE27MU
BE27RE
BE27SI
BE27SN
BE27TH
BE27WA
BE27curriculum
BE29FI
BE29MU
BE29RE
BE29SI
BE29SN
BE29TH
BE29WA
BE29curriculum
CH35FI
CH35MU
CH35RE
CH35SI
CH35SN
CH35TH
CH35WA
CH35curriculum
CH38FI
CH38MU
CH38RE
CH38SI
CH38SN
CH38TH
CH38WA
CH38curriculum


Adding a prerequisite is simple. It should affect every major that the class is in. For example, if we make CHEM 6C a prerequisite of 20C, every major that 20C is in should be affected. 

In [17]:
UCSD = read_csv("./targets/condensed.csv");
course_with_new_prereq = "MATH 20C"
course_with_new_prereq = course_from_name(UCSD,course_with_new_prereq);
affected_majors = split(course_with_new_prereq.canonical_name,",");
prev_major = "PL99";
count = 0;
for major in affected_majors
    if major != "" 
        if major[1:4] != prev_major[1:4]
            prev_major = major
            print("\n$(major[1:4]): $(major[5:end]), ");
            count += 1
        elseif major != prev_major
            prev_major = major
            print("$(major[5:end]), ");
            count += 1
        end
    end
end
println()
print("Affected plans: $count")


BE25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
BE28: FI, MU, RE, SI, SN, TH, WA, curriculum, 
BI34: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CE25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CH25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CH34: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CH36: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CS25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CS26: FI, MU, RE, SI, SN, TH, WA, curriculum, 
CS27: FI, MU, RE, SI, SN, TH, WA, curriculum, 
DS25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EC26: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EC27: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EC28: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EC37: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EN25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EN28: FI, MU, RE, SI, SN, TH, WA, curriculum, 
EN30: FI, MU, RE, SI, SN, TH, WA, curriculum, 
ES25: FI, MU, RE, SI, SN, TH, WA, curriculum, 
ES27: FI, MU, RE, SI, SN, TH, WA, curriculum, 
MA27: FI, MU, RE, SI, SN, TH, WA, curriculum, 
MA29: FI, MU